In [1]:
import tensorflow 
tensorflow.__version__

'1.15.2'

In [2]:
from __future__ import absolute_import, division, print_function, unicode_literals
#
import tensorflow as tf
#import keras.backend.tensorflow_backend as KTF
gpu_devices = tensorflow.config.experimental.list_physical_devices('GPU')

for device in gpu_devices:
    tensorflow.config.experimental.set_memory_growth(device,True)
    
gpus = tensorflow.config.experimental.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only use the first GPU
  try:
    tensorflow.config.experimental.set_visible_devices(gpus[1], 'GPU')
    logical_gpus = tensorflow.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
  except RuntimeError as e:
    # Visible devices must be set before GPUs have been initialized
    print(e)

2 Physical GPUs, 1 Logical GPU


In [3]:
tensorflow.config.experimental.set_visible_devices(gpus[1], 'GPU')

In [4]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import tensorflow.keras.layers as layers
import tensorflow.keras
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import (Reshape,Activation,Attention,MaxPool1D,Dense, Conv1D, Convolution2D, GRU, LSTM, Lambda, Bidirectional, TimeDistributed,
                          Dropout, Flatten, LayerNormalization,RepeatVector, Reshape, MaxPooling1D, UpSampling1D, BatchNormalization)
from tensorflow.keras.layers import Layer
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l1, l2
from tensorflow.keras.models import load_model
from sklearn import preprocessing
from tensorflow.keras.models import load_model
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn import linear_model
import pandas as pd
import torch
import torch.nn as nn
import random
#from keras import layers as layers
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from scipy import special
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import tensorflow as tf
import tensorflow.keras.backend as K
from tensorflow.keras.callbacks import ModelCheckpoint
#from keras_self_attention import SeqSelfAttention

from sklearn.preprocessing import StandardScaler
import os
import random
import numpy as np
import json
import matplotlib.pyplot
import pickle
from matplotlib.pyplot import imshow
from PIL import Image
from sklearn.manifold import TSNE
from tensorflow.keras.callbacks import ModelCheckpoint
from sklearn.decomposition import NMF
from scipy import io
import numpy as np
import os
import os.path
from scipy.io import loadmat
from sklearn.preprocessing import StandardScaler

In [17]:
def encode_identity_block(X, stage, block, size, n_step, drop_frac, l1_norm):
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'enconde' + str(stage) + block + '_branch'

    X_shortcut = X
    X = layers.Bidirectional(LSTM(size, return_sequences=True, dropout=drop_frac,
                                  activity_regularizer=l1(l1_norm)), input_shape=(n_step, 1))(X)

    # X = layers.BatchNormalization(axis = 1, name = bn_name_base + '2a')(X)
    X = layers.Activation('relu')(X)

    X = layers.Bidirectional(LSTM(size, return_sequences=True, dropout=drop_frac,
                                  activity_regularizer=l1(l1_norm)), input_shape=(n_step, 1))(X)

    # X = layers.BatchNormalization(axis = 1, name = bn_name_base + '2b')(X)
    X = layers.Activation('relu')(X)

    X = layers.Bidirectional(LSTM(size, return_sequences=True, dropout=drop_frac,
                                  activity_regularizer=l1(l1_norm)), input_shape=(n_step, 1))(X)

    

    X = layers.add([X, X_shortcut])
    X = layers.BatchNormalization(axis = 1, name = bn_name_base + '2c')(X)
    X = layers.Activation('relu')(X)

    return X


def decode_identity_block(X, stage, block, size, n_step, drop_frac, l1_norm):
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'decode' + str(stage) + block + '_branch'

    X_shortcut = X
    X = layers.Bidirectional(LSTM(size, return_sequences=True, dropout=drop_frac,
                                  activity_regularizer=l1(l1_norm)))(X)
    # X = layers.BatchNormalization(axis = 1, name = bn_name_base + '2a')(X)
    X = layers.Activation('relu')(X)

    X = layers.Bidirectional(LSTM(size, return_sequences=True, dropout=drop_frac,
                                  activity_regularizer=l1(l1_norm)))(X)

    # X = layers.BatchNormalization(axis = 1, name = bn_name_base + '2b')(X)
    X = layers.Activation('relu')(X)

    X = layers.Bidirectional(LSTM(size, return_sequences=True, dropout=drop_frac,
                                  activity_regularizer=l1(l1_norm)))(X)
    # 

    X = layers.add([X, X_shortcut])
    X = layers.BatchNormalization(axis = 1, name = bn_name_base + '2c')(X)
    X = layers.Activation('relu')(X)

    return X

    

def ResNet50(lr=3e-5, size=128, drop_frac=0, n_step=96, embedding=16, l1_norm=1e-5, coef=1e-2):
    X_input = layers.Input(shape=(n_step, 1))
    X = X_input

    X = encode_identity_block(X, 2, 'b', size, n_step, drop_frac, l1_norm)
    X = encode_identity_block(X, 2, 'c', size, n_step, drop_frac, l1_norm)
    X = encode_identity_block(X, 2, 'd', size, n_step, drop_frac, l1_norm)

    X = layers.Bidirectional(LSTM(size, return_sequences=False, dropout=drop_frac,
                                  activity_regularizer=l1(l1_norm)), input_shape=(n_step, 1))(X)
    #X = layers.BatchNormalization(axis=1, name='last_encode')(X)
    X = layers.Activation('relu')(X)
#    X = layers.Dense(embedding, activation='relu', name='embedding_layer', activity_regularizer=l1(l1_norm))(X)
   
    sd = 0.5 * Dense(units = embedding, name='standard_deviation',
                     activation = 'relu', activity_regularizer=l1(l1_norm))(X) 
    
    mn = Dense(units = embedding, name='mean_value',activation = 'relu', 
               activity_regularizer=l1(l1_norm))(X)
    
    def tensor_stack(layer):
        return K.random_normal(K.stack([tensorflow.shape(layer)[0], embedding]), mean=1.0) 
   
    epsilon = Lambda(lambda x: tensor_stack(x))(mn)
    
    def tensor_multi(layer):
        return tensorflow.multiply(epsilon, tensorflow.exp(layer))
    
    
    z = Lambda(lambda x: tensor_multi(x))(sd)
    X = mn + z
    
    X = Activation('relu')(X)
    
    X = layers.RepeatVector(n_step)(X)

    X = layers.Bidirectional(LSTM(size, return_sequences=True, dropout=drop_frac,
                                  activity_regularizer=l1(l1_norm)))(X)

    # X = layers.BatchNormalization(axis = 1, name = 'fires_decode')(X)
    X = layers.Activation('relu')(X)

    X = decode_identity_block(X, 2, 'b', size, n_step, drop_frac, l1_norm)
    X = decode_identity_block(X, 2, 'c', size, n_step, drop_frac, l1_norm)
    X = decode_identity_block(X, 2, 'd', size, n_step, drop_frac, l1_norm)

    X = layers.BatchNormalization(axis=1, name='batch_normal')(X)
    X = layers.TimeDistributed(Dense(1, activation='linear'))(X)
    model = Model(inputs=X_input, outputs=X, name='ResNet50')
    model.compile(Adam(lr), loss=kl_loss_1(sd,mn,coef))

    return model

In [18]:
def kl_loss_1(sd, mn, coef):
    latent_loss = -coef * tensorflow.reduce_sum(1.0 + 2.0 * sd - tensorflow.square(mn-1)
                                               - tensorflow.exp(2.0 * sd), 1)
    def loss(y_true, y_pred):
        
        img_loss = tf.reduce_sum(tf.math.squared_difference(y_pred, y_true), 1)
        return tf.reduce_mean(img_loss + latent_loss)
    
    return loss

In [19]:
Lst = ResNet50()

In [26]:
Lst.summary()

Model: "ResNet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            [(None, 96, 1)]      0                                            
__________________________________________________________________________________________________
bidirectional_60 (Bidirectional (None, 96, 256)      133120      input_6[0][0]                    
__________________________________________________________________________________________________
activation_61 (Activation)      (None, 96, 256)      0           bidirectional_60[0][0]           
__________________________________________________________________________________________________
bidirectional_61 (Bidirectional (None, 96, 256)      394240      activation_61[0][0]              
___________________________________________________________________________________________

In [21]:
pieoresponse = np.load('pizoresponse.npy')

In [22]:
filepath="Piezoresponse_Model/weights-lstm_vae+{epoch:02d}.hdf5"

In [23]:
checkpointer = ModelCheckpoint(filepath, monitor='loss', 
                               verbose=0, save_best_only=True, save_weights_only = True)

In [25]:
Lst.fit(x= pieoresponse, y = pieoresponse,epochs=10,
              batch_size=128,shuffle = True,callbacks=[checkpointer])

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Train on 3600 samples
Epoch 1/10


ResourceExhaustedError: 2 root error(s) found.
  (0) Resource exhausted: OOM when allocating tensor with shape[96,128,128] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[{{node bidirectional_79/forward_lstm_79_1/TensorArrayStack/TensorArrayGatherV3}}]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

	 [[loss/add/_2983]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

  (1) Resource exhausted: OOM when allocating tensor with shape[96,128,128] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[{{node bidirectional_79/forward_lstm_79_1/TensorArrayStack/TensorArrayGatherV3}}]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions fo